# WEBINAR - Przetwarzanie języka polskiego ze spaCy, 26 października 2020

NLPDay 2020

Ryszard Tuora


2 modele do języka polskiego w spaCy:

IPI PAN - bardziej rozbudowany, wolniejszy, bardziej złożona instalacja - http://zil.ipipan.waw.pl/SpacyPL

model oficjalny - prostszy, znacznie szybszy, prosta instalacja - https://spacy.io/models/pl

model IPI PAN dla języka polskiego

składa się z:
- taggera morfosyntaktycznego
- lematyzatora
- parsera zależnościowego
- komponentu NER
- flexera (komponentu do fleksji)

In [ ]:
# Przygotowanie środowiska, komendy linux
# instalacja Morfeusza 2
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja spaCy

!python3 -m pip install spacy

# 1. instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz' pl_spacy_model_morfeusz-0.1.0.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.0.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

# 2. instalacja oficjalnego modelu spaCy
!python3 -m spacy download pl_core_news_lg

# dodatkowe zależności:
!python3 -m pip install tqdm
!python3 -m pip install networkx

In [1]:
### PYTHON 3
# ładowanie modelu
import spacy
import requests

nlp = spacy.load("pl_spacy_model_morfeusz")
#nlp = spacy.load("pl_core_news_lg")

# Część zero - Tokenizacja i reprezentacja tekstów

Wejściem do modelu są łańcuchy tekstowe (stringi), na wyjściu dostajemy obiekt Doc reprezentujący struktury wykryte w tekście przez pełen potok (pipeline). Pierwszym krokiem który musi być wykonany aby przetworztć tekst, jest tokenizacja, czyli podział tekstu na tokeny/segmenty. Tokeny w większości przypadków odpowiadają słowom "od spacji do spacji". Ale warto zwrócić uwagę na kilka przypadków odstających od takiej prostej reguły.

Wynikiem potoku spaCy jest obiekt Doc, który składa się z obiektów Token.

In [87]:
txt = "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry)."
split = txt.split()
doc = nlp(txt)
print("    spaCy     vs.   .split()\n")
for i in range(max([len(split), len(doc)])):
    try:
        tok1 = doc[i]
    except IndexError:
        tok1 = ""
    try:
        tok2 = split[i]
    except IndexError:
        tok2 = ""
    print("{0:2}. {1:15} {2:15}".format(i, tok1.orth_, tok2))

    spaCy     vs.   .split()

 0. Chciał          Chciałby,      
 1. by              żebym          
 2. ,               pojechał       
 3. żeby            do             
 4. m               miasta         
 5. pojechał        z              
 6. do              zielono-żółto-białą
 7. miasta          flagą          
 8. z               (np.           
 9. zielono         Zielonej       
10. -               Góry).         
11. żółto                          
12. -                              
13. białą                          
14. flagą                          
15. (                              
16. np                             
17. .                              
18. Zielonej                       
19. Góry                           
20. )                              
21. .                              


spaCy pozwala na zapisywanie przetworzonych dokumentów w formacie JSON

In [88]:
import json

doc_json = doc.to_json()
json_string = json.dumps(doc_json, indent=2, ensure_ascii=False)
print(json_string)

{
  "text": "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry).",
  "ents": [
    {
      "start": 68,
      "end": 81,
      "label": "placeName"
    }
  ],
  "sents": [
    {
      "start": 0,
      "end": 83
    }
  ],
  "tokens": [
    {
      "id": 0,
      "start": 0,
      "end": 6,
      "pos": "VERB",
      "tag": "praet",
      "dep": "ROOT",
      "head": 0
    },
    {
      "id": 1,
      "start": 6,
      "end": 8,
      "pos": "PART",
      "tag": "qub",
      "dep": "aux:cnd",
      "head": 0
    },
    {
      "id": 2,
      "start": 8,
      "end": 9,
      "pos": "PUNCT",
      "tag": "interp",
      "dep": "punct",
      "head": 5
    },
    {
      "id": 3,
      "start": 10,
      "end": 14,
      "pos": "SCONJ",
      "tag": "comp",
      "dep": "mark",
      "head": 5
    },
    {
      "id": 4,
      "start": 14,
      "end": 15,
      "pos": "AUX",
      "tag": "aglt",
      "dep": "aux:clitic",
      "head": 5
    },
    {
   

# Część pierwsza - reprezentacje wektorowe

Sercem modelu jest reprezentacja wektorowa słów, 500 tys. wektorów wyekstrahowanych z embeddingów word2vec KGR10.

### Bardzo popularny przykład "arytmetyki słów":
znaczenie słów jest reprezentowane przez wektory, dla których mamy zdefiniowane operacje matematyczne. Możemy więc odjąć od znaczenia słowa "królowa", znaczenie słowa "kobieta", i dodać doń znaczenie słowa "mężczyzna", licząc iż wektor będący wynikiem takiego działania odpowiada słowu "król". W praktyce wektor taki najprawdopodobniej nie ma interpretacji, ale możemy znaleźć najbliższy wektor który ma jakąkolwiek interpretację przeszukując słownik.

In [89]:
# od wersji 2.3 wektory są ładowane "leniwie"
# musimy więc najpierw wymusić załadowanie wszystkich wektorów
print(len(nlp.vocab))
for s in nlp.vocab.vectors:
    _ = nlp.vocab[s]
print(len(nlp.vocab))

In [90]:
from scipy import spatial
 
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['mężczyzna'].vector
woman = nlp.vocab['kobieta'].vector
queen = nlp.vocab['królowa'].vector
king = nlp.vocab['król'].vector
 

maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
sorted_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in sorted_similarities[:10]])

['książę', 'król', 'królewicz', 'monarcha', 'hrabia', 'władca', 'Książę', 'arcyksiążę', 'książe', 'Arcyksiążę']


In [91]:
w1 = "pies"
w2 = "psami"
w3 = "zwierzę"
w4 = "buldog"
w5 = "obroża"
w6 = "smycz"
w7 = "marchewka"
w8 = "słońce"

def similarity(w1, w2):
    w1_lex = nlp.vocab[w1]
    w2_lex = nlp.vocab[w2]
    if w1_lex.has_vector and w2_lex.has_vector:
        sim = w1_lex.similarity(w2_lex)
        print("{} vs. {} -> {}".format(w1, w2, sim))
    else:
        print("Jedno ze słów nie ma reprezentacji wektorowej.")

for w in [w2, w3, w4, w5, w6, w7, w8]:
    similarity(w1, w)

pies vs. psami -> 0.6705560088157654
pies vs. zwierzę -> 0.684103786945343
pies vs. buldog -> 0.8052963018417358
pies vs. obroża -> 0.5760606527328491
pies vs. smycz -> 0.5837645530700684
pies vs. marchewka -> 0.4573368728160858
pies vs. słońce -> 0.2709254324436188


## Zadanie 1:

nlp.vocab może być traktowany jako iterator: znajdują się w nim obiekty klasy lexeme, nie wszystkim obiektom lexeme przypisane są wektory - można to poznać po atrybucie .has_vector. Napisz funkcję thesaurus(word) która dla podanego słowa znajduje dziesięć najbardziej podobnych słów. Możesz skorzystać z metody .similarity zdefiniowanej dla obiektów lexeme, która jako argument przyjmuje drugi obiekt lexeme.


### spaCy umożliwia liczenie podobieństwa między słowami 
jest ono proporcjonalne do cosinusa kąta wektorami je reprezentującymi. Odpowiednia funkcja jest zdefiniowana dla leksemów (tutaj oznaczają one słowa z pominięciem kontekstu).

In [51]:
# TODO
def thesaurus(word):
    synonyms = []
    minimum = 0
    w_lex = nlp.vocab[word]
    for candidate in nlp.vocab:
        if candidate.has_vector and candidate != w_lex:
            similarity = w_lex.similarity(candidate)
            if similarity > minimum:
                synonyms.append((similarity, candidate.orth_))
            if len(synonyms) >10:
                synonyms = sorted(synonyms, reverse = True)
                synonyms = synonyms[:-1]
                minimum = synonyms[-1][0]

    return synonyms

syn = (thesaurus("pies"))
for x in syn:
    print(x)

(0.9248929, 'piesek')
(0.9197937, 'szczeniak')
(0.9130602, 'Pies')
(0.9057689, 'kot')
(0.9051197, 'wilczur')
(0.9018397, 'zwierzak')
(0.9007868, 'psiak')
(0.8871712, 'owczarek')
(0.88386726, 'czworonóg')
(0.87603015, 'jamnik')


spaCy konstruuje także wektory dla zdań, i dokumentów. Wektor dla dokumentu jest dostępny w atrybucie .vector obiektu doc, i jest równy uśrednionemu wektorowi tokenów z tego dokumentu.

In [116]:
document_vector = doc.vector
mean_document_vector = sum([tok.vector for tok in doc])/len(doc)
print(document_vector == mean_document_vector)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


## Dezambiguacja semantyczna

Podstawowym narzędziem służącym do określenia który z sensów wyrazu wieloznacznego występuje w tekście, jest algorytm Leska. W wersji tradycyjnej, opiera się o reprezentację BoW, i zdefiniowane dla nich miary podobieństwa, ale możemy także uogólnić go do postaci działającej dla reprezentacji word2vec.

Algorytm porównuje kontekst w jakim znajduje się docelowe słowo, z glosą, wydobytą z bazy sensów. Tutaj miarą dopasowania będzie podobieństwo cosinusowe.

## Zadanie 2:
W zmiennej wsd_data umieszczono słownik sensów dla wyrazów "zamek", "igła", "pilot", oraz "klucz", wyekstrahowany ze Słowosieci.

Przygotuj funkcję, która wykorzysta reprezentacje wektorowe zdań (lub dokumentów), do implementacji algorytmu Leska. dla każdego z przykładów, i wskazanego słowa do dezambiguacji, powinna zwrócić sens, o który chodzi w danym kontekście. Skorzystaj z metody .similarity definiowanej dla obiektów typu doc, oraz sent.


In [45]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/wsd_data.json").text

wsd_data = json.loads(txt)
for lemma in wsd_data:
    for variant, gloss in wsd_data[lemma]:
        print(lemma, variant, gloss)
        
sent_to_disamb1 = "Początkowo zamek miał kształt zbliżony do owalu, \
                   który był otoczony grubymi na dwa metry murami \
                   obwodowymi z blankami, na jego wewnętrznym dziedzińcu\
                   prawdopodobnie znajdowały się drewniane zabudowania."

sent_to_disamb2 = "Pielęgniarka nie mogła znaleźć żyły, igła co raz rozcinała skórę."

# TODO

def desambiguate(text, word):
    glosses = wsd_data[word]
    target_doc = nlp(text)
    ranking = []
    for v, g in glosses:
        doc = nlp(g)
        sim = target_doc.similarity(doc)
        ranking.append((sim, g))
    ranking = sorted(ranking, reverse=True)
    return ranking[0]

print(desambiguate(sent_to_disamb1, "zamek"))
print(desambiguate(sent_to_disamb2, "igła"))

zamek 1 warowna budowla mieszkalna, rezydencja pana, króla, księcia lub magnata.
zamek 2 mechanizm lub urządzenie do zamykania drzwi, szuflad, walizek.
zamek 6 zapięcie przy ubraniu, suwak, ekler.
zamek 4 mechanizm broni palnej służący do zamykania na czas wystrzału i otwierania po strzale tylnej części lufy.
zamek 7 zagranie taktyczne w hokeju; zamknięcie przeciwnika w jego tercji lodowiska/boiska.
zamek 3 urządzenie do łączenia lub zabezpieczania w ustalonym położeniu elementów maszyny.
zamek 5 blokada w informatyce.
igła 5 liść o blaszce, która jest podłużna, wąskaj, sztywna i ostra.
igła 3 iglica - zwieńczenie wieży lub hełmu w formie wysmukłego ostrosłupa lub stożka.
igła 7 iglica - część urządzenia (np. igła cyrkla, igła od wrzeciona).
igła 1 cienki, wydrążony w środku pręcik, używany jako narzędzie medyczne, np. do zastrzyków, pobierania krwi, kroplówki.
igła 4 cienki metalowy pręcik z uszkiem służący do szycia.
igła 2 rodzaj dużej igły do dziania sieci rybackich.
pilot 3 pierws

# Część druga - Tagowanie morfosyntaktyczne
korzystamy z tagsetu NKJP
Nasz tagger to słownikowy tagger Morfeusz2 + dezambiguacja za pomocą neuronowego Toyggera (biLSTM)

Każdy token t ma trzy interesujące nas atrybuty: 
- t.tag_ : klasa gramatyczna według polskiego tagsetu NKJP (http://nkjp.pl/poliqarp/help/ense2.html)
- t.pos_ : klasa gramatyczna według międzynarodowego tagsetu UD (mapowana z NKJP)
- t._.feats : customowy atrybut odpowiadający cechom morfosyntaktycznym (np. rodzajowi gramatycznemu, lub liczbie), poszczególne wartości cech są oddzielone dwukropkiem

In [94]:
txt = "Nornica prowadzi zmierzchowo-nocny tryb życia, ale wychodzi również za dnia w poszukiwaniu pokarmu."
doc = nlp(txt) # przetworzenie textu przez pipeline, na wyjściu dostajemy iterowalny obiekty klasy Doc, przechowujący tokeny

print("{0:15} {1:8} {2:6} {3:15}\n".format(".orth_", "NKJP", "UD POS", "._.feats"))
for t in doc:
    print("{0:15} {1:8} {2:6} {3:15}".format(t.orth_, t.tag_, t.pos_, t._.feats)) # wypisujemy interpretację morfosyntaktyczną każdego tokenu

.orth_          NKJP     UD POS ._.feats       

Nornica         subst    NOUN   sg:nom:f       
prowadzi        fin      VERB   sg:ter:imperf  
zmierzchowo     adja     ADJ                   
-               interp   PUNCT                 
nocny           adj      ADJ    sg:acc:m3:pos  
tryb            subst    NOUN   sg:acc:m3      
życia           subst    NOUN   sg:gen:n       
,               interp   PUNCT                 
ale             conj     CCONJ                 
wychodzi        fin      VERB   sg:ter:imperf  
również         qub      PART                  
za              prep     ADP    gen            
dnia            subst    NOUN   sg:gen:m3      
w               prep     ADP    loc:nwok       
poszukiwaniu    ger      NOUN   sg:loc:n:imperf:aff
pokarmu         subst    NOUN   sg:gen:m3      
.               interp   PUNCT                 


## Zadanie 3:

Charakterystyka stylów literackich wiąże się z proprocjami części mowy w tekście.

![alt text](https://github.com/ryszardtuora/workshop_resources/raw/main/czesci_mowy.png)

Źródło: Irena Kamińska-Szmaj, *Różnice leksykalne między stylami funkcjonalnymi polszczyzny pisanej: Analiza statystyczna na materiale słownika frekwencyjnego*, 1990.

Przetwórz tekst znajdujący się w zmiennej txt, oblicz proporcję czasowników (używając tagów UD), na podstawie tego oszacuj gatunek do którego należy tekst.

In [95]:
import requests

txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/1.txt").text

# TODO
doc = nlp(txt)
nouns = 0
verbs = 0
tokens = 0
non_interp = 0
for t in doc:
    tokens += 1
    if t.pos_ != "PUNCT":
        non_interp +=1
    if t.pos_ == "NOUN":
        nouns += 1
    if t.pos_ == "VERB":
        verbs += 1
    
print("Stosunek czasowników do rzeczowników: ", verbs/nouns)
print("Proporcja czasowników: ", verbs/non_interp)
print("Proporcja rzeczowników: ", nouns/non_interp)

Stosunek czasowników do rzeczowników:  0.6528925619834711
Proporcja czasowników:  0.19494139420111042
Proporcja rzeczowników:  0.2985811227637261


### Fleksja

Flexer pozwala na odmianę pojedynczych tokenów, do pożądanej charakterystyki morfologicznej. Argumentami do flexera jest słowo do odmiany (string, lub lepiej otagowany token), i przedzielony dwukropkami string znaczników morfosyntaktycznych.

Flexer umożliwia np. wypełnianie szablonów tekstów.

In [96]:
filizanka = "filiżanka"
flexer = nlp.get_pipe("flexer")

tmpl1 = "Szukam szarej, ceramicznej {}.".format(flexer.flex(filizanka, "gen"))
tmpl2 = "Marzę o szarej, ceramicznej {}.".format(flexer.flex(filizanka, "loc"))
tmpl3 = "Szukam kompletu ceramicznych {}.".format(flexer.flex(filizanka, "gen:pl"))
print(tmpl1)
print(tmpl2)
print(tmpl3)

Szukam szarej, ceramicznej filiżanki.
Marzę o szarej, ceramicznej filiżance.
Szukam kompletu ceramicznych filiżanek.


#### Flexer pozwala także na odmianę fraz wielowyrazowych (multi-word expressions - MWE).

W tym celu możemy do niego podać string, lub token odpowiadający głowie frazy.
Korzystamy z metody .flex_mwe()

In [97]:
filizanka2 = "biała, porcelanowa filiżanka z Chin"

tmpl1 = "Szukam {}.".format(flexer.flex_mwe(filizanka2, "gen"))
tmpl2 = "Marzę o {}.".format(flexer.flex_mwe(filizanka2, "loc"))
tmpl3 = "Szukam kompletu {}.".format(flexer.flex_mwe(filizanka2, "gen:pl"))

print(tmpl1)
print(tmpl2)
print(tmpl3)

Szukam białej, porcelanowej filiżanki z Chin.
Marzę o białej, porcelanowej filiżance z Chin.
Szukam kompletu białych, porcelanowych filiżanek z Chin.


## Zadanie 4

Zaprojektuj sposób formułowania znaczników morfosyntaktycznych dla luk w szablonach, który określa jaką formę muszą przyjąć frazy, by móc wypełnić te luki.
Zaprojektuj funkcję, która pobiera jako argumenty 1. szablon z tak oznakowaną luką, 2. frazę do wypełnienia luki, która odmienia wskazaną frazę, i wkleja ją w miejsce luki. 
Możesz założyć, że funkcja ma działać wtedy, i tylko wtedy, gdy w szablonie znajduje się dokładnie jedna luka.

In [16]:
template = "Przyglądam się ___." # w miejsce ___ wstaw zaprojektowany znacznik, który pozwoli na poprawne
                                 # uzupełnienie luki
phrase = "czarne, skórzana torebka ze sprzączką"
#TODO
import re

gap_tag = "#gen:loc#"
gap_regex = re.compile("#[^#]+#")
template = "Przyglądam się #dat:pl#."
phrase = "czarne, skórzana torebka ze sprzączką"

def fill_template(template, phrase):
    match = gap_regex.search(template)
    tag = match.group(0)
    morpho = tag.strip("#")
    flexed = flexer.flex_mwe(phrase, morpho)
    filled = template.replace(tag, flexed)
    return filled

print(fill_template(template, phrase))

Przyglądam się czarnym, skórzanym torebkom ze sprzączką.


# Część trzecia - Lematyzacja i własności leksykalne
Nasz model umożliwia słownikową lematyzację przy pomocy Morfeusza, do dezambiguacji (tutaj np. rozróżnienia między "Głos zabrały mamy dzieci."-> "mama" i "My mamy samochód." -> "mieć" służy output taggera).

Lematyzacja pozwala redukować redundancję informacyjną, i ułatwiać zadania takie jak streszczanie, i przeszukiwanie.

Każdy z tokenów dodatkowo jest oznaczony ze względu na pewne własności leksykalne, np. :
- t.lemma_ - lemat - forma reprezentatywna danego leksemu
- t.is_stop - słowo należy do stoplisty (listy słów występujących najczęściej, a więc najmniej istotnych semantycznie)
- t.is_oov - słowo znajduje się poza słownikiem, i.e. embeddingami wykorzystanymi w modelu
- t.like_url - token ma strukturę url-a
- t.like_num - token jest liczbą
- t.is_alpha - token składa się tylko ze znaków alfabetycznych
- t.rank - miejse w rankingu częstości słów

In [98]:
txt = "Zdenerwowany gen. Leese mówił przez telefon swym podwładnym walczącym pod Monte Cassino, że rozmawia z nimi ze schronu."
doc = nlp(txt)
print("{0:16} {1:16} {2:5} {3:5} {4:20}\n".format("forma", "lemat", "OOV", "STOP", "Częstość"))
for t in doc:
    print("{0:16} {1:16} {2:5} {3:5} {4:20}".format(t.orth_, t.lemma_, t.is_oov, t.is_stop, t.rank)) # orth_ to atrybut odpowiadający formie słowa występującej w tekście

forma            lemat            OOV   STOP  Częstość            

Zdenerwowany     zdenerwować          0     0               117789
gen              generał              0     0                 4239
.                .                    0     0                    1
Leese            Leese                1     0 18446744073709551615
mówił            mówić                0     0                  536
przez            przez                0     1                   32
telefon          telefon              0     0                 1629
swym             swój                 0     0                 1994
podwładnym       podwładny            0     0                69725
walczącym        walczyć              0     0                58343
pod              pod                  0     1                   84
Monte            Monte                0     0                15050
Cassino          Cassino              0     0                33875
,                ,                    0     0                

## Zadanie 5:

Przetwórz tekst spod zmiennej txt, przekonwertuj go do listy lematów, usuń słowa ze stoplisty, oraz interpunkcję, wypisz dziesięć najczęściej pojawiających się lematów. Wypróbuj także opcję w której uwzględniamy tylko rzeczowniki.


In [57]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/2.txt").text

#TODO
doc = nlp(txt)
words = {}
for t in doc:
    if not (t.is_stop or t.tag_ == "interp"):
        if t.lemma_ in words:
            words[t.lemma_] +=1
        else:
            words[t.lemma_] = 1
listed = [(words[w],w) for w in words]
srt = sorted(listed, reverse = True)
print(srt[0:10])

[(8, 'miejsce'), (4, 'praca'), (4, 'odbyć'), (4, 'dzień'), (4, 'Podhale'), (3, 'złoty'), (3, 'trwać'), (3, 'spowodować'), (3, 'pierwszy'), (3, 'otworzyć')]


# Część czwarta - parsowanie zależnościowe

spaCy zawiera parser zależnościowy oparty o metodologię transition-based dependency parsing. 
Interesują nas tu cztery atrybuty:
 - t.head - link do tokenu będącego nadrzędnikiem tokenu t
 - t.dep_ - etykieta opisująca rodzaj zależności
 - t.subtree - generator opisujący poddrzewo rozpięte przez token t
 - t.children - generator opisujący wszystkich bezpośrednich potomków tokenu t
 - t.ancestors - generator opisujący wszystkie przechodnie nadrzędniki tokenu t


Opis systemu etykiet: https://universaldependencies.org/u/dep/all.html

In [99]:
txt = "Pierwsza wzmianka o Gdańsku pochodzi ze spisanego po łacinie w 999 Żywotu świętego Wojciecha."
doc = nlp(txt)

import pandas as pd

table = []
for tok in doc:
    tok_dic = {"form": tok.orth_, "label": tok.dep_, "head": tok.head.orth_, "subtree": list(tok.subtree), "ancestors": list(tok.ancestors)}
    table.append(tok_dic)
df = pd.DataFrame(table)
print(df.to_string())

         form    label       head                                                                                                         subtree                                 ancestors
0    Pierwsza     amod   wzmianka                                                                                                      [Pierwsza]                      [wzmianka, pochodzi]
1    wzmianka    nsubj   pochodzi                                                                                [Pierwsza, wzmianka, o, Gdańsku]                                [pochodzi]
2           o     case    Gdańsku                                                                                                             [o]             [Gdańsku, wzmianka, pochodzi]
3     Gdańsku     nmod   wzmianka                                                                                                    [o, Gdańsku]                      [wzmianka, pochodzi]
4    pochodzi     ROOT   pochodzi  [Pierwsza, wzmianka, o, G

### spaCy posiada wbudowaną wizualizację drzew zależnościowych

In [100]:
from spacy import displacy

displacy.render(doc, jupyter = True)

### Poniższa funkcja służy łatwej wizualizacji podstawowych własności tokenów z danego tekstu

In [101]:
import pandas as pd

def table(doc):
    table = []
    for tok in doc:
        tok_dic = {"form": tok.orth_, "lemma": tok.lemma_, "tag": ":".join([tok.tag_, tok._.feats]), "dep_label": tok.dep_, "dep_head": tok.head.orth_}
        table.append(tok_dic)
    return pd.DataFrame(table)

txt = "Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.\
 Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych \
 wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii \
 (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka\
  będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki \
  polskiej. Nożyce między oboma krajami będą się więc rozwierać."

doc = nlp(txt)

tab = table(doc)

print(tab.to_string()) # prosty hack na wypisanie całej tabeli

               form           lemma                     tag    dep_label         dep_head
0         Wiadomość       wiadomość          subst:sg:nom:f        nsubj      symboliczna
1              jest             być       fin:sg:ter:imperf          cop      symboliczna
2       symboliczna     symboliczny        adj:sg:nom:f:pos         ROOT      symboliczna
3                 ,               ,                 interp:        punct          oznacza
4               ale             ale                   conj:           cc          oznacza
5           oznacza        oznaczać       fin:sg:ter:imperf         conj      symboliczna
6               też             też                    qub:  advmod:emph          oznacza
7          początek        początek         subst:sg:acc:m3         iobj          oznacza
8     długotrwałego     długotrwały       adj:sg:gen:m3:pos         amod           trendu
9            trendu           trend         subst:sg:gen:m3     nmod:arg         początek
10        

### Czasami interesują nas większe całości niż pojedyncze tokeny, 
np. rzeczowniki często łączą się w frazy rzeczownikowe, żeby znajdować takie całości w tekście możemy korzystać z Matchera.

In [102]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
matcher.add("NounAdpNoun", None, pattern) # nazwa, funkcja, wzór

for match_id, start, end in matcher(doc):
    toks = doc[start:end]
    print(toks) # obiekt typu Span

Dochód na mieszkańca
mieszkańca z uwzględnieniem


### Lematyzacja jednostek wielowyrazowych

Flexer pozwala także na prostą lematyzację jednostek wielowyrazowych.

In [103]:
print("Flexer vs. konkatenacja lematów:\n")
for match_id, start, end in matcher(doc):
    phrase = doc[start:end]
    phrase_text = phrase.text
    toks = [t for t in phrase]
    lemmas = [t.lemma_ for t in toks]
    lemma_concat = " ".join(lemmas)
    print("{} vs. {}".format(flexer.lemmatize_mwe(phrase_text), lemma_concat))



Flexer vs. konkatenacja lematów:

dochód na mieszkańca vs. dochód na mieszkaniec
mieszkaniec z uwzględnieniem vs. mieszkaniec z uwzględnić


## Zadanie 6

Znajdź wszystkie frazy postaci przymiotnik - rzeczownik, i wypisz je w formie występującej w tekście, oraz formie zlematyzowanej.


In [108]:
# TODO
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "ADJ"}, {"POS": "NOUN"}]
matcher.add("AdjNoun", None, pattern) # nazwa, funkcja, wzór

for match_id, start, end in matcher(doc):
    phrase_text = doc[start:end].text
    lemmatized = flexer.lemmatize_mwe(phrase_text)
    print(lemmatized)

długotrwały trend
realna moc
nabywczy walut
ten rok
portugalska gospodarka


#### Matcher pozwala także na korzystanie z innych własności tokenów, np. lematów.

### Parser zależnościowy pozwala także na dzielenie dokumentów na zdania w sposób bardziej inteligentny, niż posługując się regułami interpunkcji.
Za zdanie uważamy nieprzerwaną sekwencję tokenów które są powiązane relacjami zależnościowymi.
Zdania są zapisane w atrybucie doc.sents dokumentu.

In [110]:
for s in doc.sents:
    print(s)

displacy.render(doc, jupyter = True)

Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.
Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii (33 665 dolarów).
Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki polskiej.
Nożyce między oboma krajami będą się więc rozwierać.


# Część czwarta - Rozpoznawanie jednostek nazewniczych (NER)
#### Nasz model do spaCy wykorzystuje 6 rodzajów etykiet:
- placeName - miejsca antropogeniczne, np. Dania, Londyn
- geogName - naturalne miejsca geograficzne, np. Tatry, Kreta
- persName - imiona i nazwiska osób, np. J. F. Kennedy, gen. Maczek
- orgName - nazwy organizacji, np. NATO, Unia Europejska
- date - daty, np. 22 marca 1999, druga połowa kwietnia
- time - czas, np. 18:55, pięć po dwunastej

#### Nie pozwala na wykrywanie zagnieżdżonych jednostek nazewniczych, np. [placeName: **aleja** [persName: **Piłsudskiego**]]

Wykryte wzmianki są przechowywane w atrybucie doc.ents dokumentu, każda z tych wzmianek ma atrybut ent.label_, w którym przechowywana jest jej etykieta.

In [111]:
print(doc, "\n\n")
for e in doc.ents:
    print(e, e.label_)


Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu. Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki polskiej. Nożyce między oboma krajami będą się więc rozwierać. 


2019 r. date
Rzeczpospolitej Polskiej placeName
Portugalii placeName
Fundusz orgName
portugalska placeName
polskiej placeName


### displaCy pozwala także na wizualizację jednostek nazewniczych

In [112]:
displacy.render(doc, style="ent", jupyter = True)

W obecnym modelu NER, uwzględnione są "uniwersalne" kategorie jednostek nazewniczych, jednak w zależności od zastosowania będziemy najprawdopodobniej potrzebowali innych kategorii - np. osobnej kategorii dla nazw aktów prawodawczych, lub kwot i walut.

Bez problemu można zastąpić domyślny model własnym, wytrenowanym przez CLI spaCy na własnych oznakowanych danych. Więcej informacji tu: https://spacy.io/api/cli#train

## Zadanie 7:

W zmiennej txt znajduje się dokument historyczny, wydobądź z niego wszystkie *zdania* zawierające daty, i po kolei je wypisz, rozważ możliwe sposoby automatycznego szeregowania dat wyrażonych w różny sposób (po ewentualnym sprowadzeniu ich do kanonicznej postaci). Rozwiązanie tego problemu jest trudne, i istnieją do niego odrębne narzędzia, umożliwiają one np. automatyczną rekonstrukcję chronologii wydarzeń.

In [113]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/3.txt").text

doc = nlp(txt)
events = []
for s in doc.sents:
    ents = s.ents
    labs = [e.label_ for e in ents]
    if "date" in labs:
        events.append(s)

for s in events:
    print(s)

Już na początku 1943 roku, kiedy wojska niemieckie prowadziły walki w okrążeniu pod Stalingradem i po decydującym zwycięstwie w tej bitwie 2 lutego, Armia Czerwona przystąpiła do generalnej kontrofensywy na południowym odcinku frontu wschodniego od rejonu Woroneża do północnego Kaukazu.
Nad górnym biegiem Donu 13 stycznia 1943 roku oddziały Frontu Woroneskiego rozpoczęły operację ostrogożsko-rossoszańską skierowaną przeciwko niemieckiej Grupie Armii „B”.
24 stycznia, na trzy dni przed zlikwidowaniem ostatnich okrążonych jednostek państw Osi pod Rossoszą, Front Woroneski, wspólnie z Frontem Briańskim na jego prawym skrzydle, przystąpił do operacji woronesko-kastornojeskiej.
Podczas tej operacji siłom tych dwóch frontów udało się otoczyć duże zgrupowanie wojsk niemiecko-węgierskich między Kastornoje a Woroneżem, z których część została zniszczona, a innym udało się na początku lutego przerwać okrążenie i przedostać dalej na zachód.
2 lutego wojska Frontu Woroneskiego wspólnie z jedną arm

# Część Piąta - Zastosowania Różne

W tej części pokażemy kilka przykładów wykorzystania spaCy razem z innymi bibliotekami

### Streszczanie

Streszczanie w najprostszej formie polega na wybraniu najistotniejszych zdań z dokumentu.
W materiałach znajduje się zescrapowany z Wikipedii artykuł nt. Polityki Gospodarczej, długości 92 zdań.

In [120]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/workshop_resources/main/polityka_gospodarcza.txt").text

doc = nlp(txt)
sents = list(doc.sents)
n_sents = len(sents)
print(n_sents)

92


### Jednym ze sposobów punktowania zdań jest wykorzystanie algorytmu PageRank

W tym celu musimy zinterpretować macierz podobieństwa zdań w dokumencie, jako macierz sąsiedztwa w grafie. Podobieństwo zaś, liczymy jako podobieństwo cosinusowe zdań do siebie.

In [123]:
import numpy
from sklearn.metrics import pairwise
import networkx as nx


# przygotowanie macierzy podobieństwa
vex = numpy.array([sent.vector for sent in sents])
sim_matrix = 1 - pairwise.cosine_distances(vex, vex)

# zastosowanie algorytmu TextRank
sim_graph = nx.from_numpy_matrix(sim_matrix)
sent_scores = nx.pagerank(sim_graph)


# przygotowanie wyników
ranked_sents = sorted(((sent_scores[i], i, sent) for i, sent in enumerate(sents)), reverse=True)
top_5 = ranked_sents[:5]
ordered_top_5 = sorted(top_5, key=lambda x:x[1])
summary_sents = [e[2].text for e in ordered_top_5]
summary = " ".join(summary_sents)
print(summary)

Polityka gospodarcza – to subdyscyplina ekonomii opisująca i wyjaśniająca sposoby świadomego oddziaływania państwa na gospodarkę, za pomocą określonych narzędzi (instrumentów) i środków, dla osiągnięcia celów założonych przez podmioty polityki gospodarczej (tj. władze), w otoczeniu uwarunkowań doktrynalnych (tj. w oparciu o daną teorię ekonomiczną), wewnętrznych (związanych z danym krajem) i zewnętrznych (poza nim). W tym zakresie do najważniejszych sfer działalności państwa zaliczyć należy politykę zagraniczną, kształtującą uwarunkowania zewnętrzne, politykę edukacyjną, naukową i inwestycyjną, które wpływają na uwarunkowania dotyczące zasobów, a także działania systemowe i administracyjne, mające na celu zdeterminowanie ustroju państwa. Istnieją różne modele opisujące związek pomiędzy polityką gospodarczą a polityką społeczną, są to: W pierwszym modelu polityka gospodarcza jest nadrzędna w stosunku do polityki społecznej i zajmuje się wytwarzaniem. Istnieje wiele sposobów na zmierzeni